In [11]:
import numpy as np
import pandas as pd 
import tensorflow as tf
import pickle
from tensorflow.keras.models import load_model

In [26]:
# Load the trained model , scaler , onhot encoder 
model = load_model('model.h5')

# Load the encoders and scalers
with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender=pickle.load(file)

with open('onehot_encoder.pkl','rb') as file:
    onehot_encoder=pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler=pickle.load(file) 

In [13]:
# sample data 
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [14]:
# onehot encode geography
geo_encode=onehot_encoder.transform([[input_data['Geography']]]).toarray()
geo_encode_df = pd.DataFrame(geo_encode,columns=onehot_encoder.get_feature_names_out(['Geography']))
geo_encode_df.head()

d:\GenAI\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [15]:
input_data_df=pd.DataFrame([input_data])
input_data_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [16]:
# encode categorical variable
input_data_df['Gender'] = label_encoder_gender.transform(input_data_df['Gender'])
input_data_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [17]:
# concatination one hot encoded
input_data_df = pd.concat([input_data_df.drop("Geography",axis=1),geo_encode_df],axis=1)
input_data_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [27]:
# scaling the input data 
input_scaled = scaler.transform(input_data_df)
input_scaled

array([[-0.47154541,  0.90911166,  0.09477172, -0.69844549, -0.29010416,
         0.80510537,  0.63367318,  0.95214374, -0.84805047,  0.98019606,
        -0.57581067, -0.56349184]])

In [28]:
# predication churn 
predication = model.predict(input_scaled)
predication

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 260ms/step


array([[0.0563565]], dtype=float32)

In [29]:
prediction_prob = predication[0][0]


In [30]:
prediction_prob

np.float32(0.056356505)

In [32]:
prob = float(prediction_prob)
print(prob)

0.05635650455951691


In [33]:
predication_percentage = prob*100
predication_percentage

5.635650455951691

In [36]:
if prob > 0.5:
    print("Customer likely to churn")
else:
    print("Customer likely to stay")


Customer likely to stay
